In [1]:
import numpy as np
import mir_eval.transcription
import os
from IPython.display import display, Markdown

import transcribe_factorization as tf
import evaluate_transcription as et
import STFT
import pandas as pd
import note_seq as ns

In [2]:
def load_ref_in_array_midi(ref_path, time_limit=None):
  
    truth_array = []
    note_seq = ns.midi_file_to_note_sequence(ref_path).notes
    for i in range(len(note_seq)):
        note = note_seq[i]
        line_to_array = [note.start_time, note.end_time, note.pitch]

        if(time_limit != None) and (float(line_to_array[0]) > time_limit):
            # if onset > time_limit (note outside of the cropped excerpt)
            break
        else:
            truth_array.append(line_to_array)

    return truth_array


In [5]:
def printmd(string):
    display(Markdown(string))

def compute_scores_database_maestro(piano_type, T, H_normalization = False, adaptative_threshold = True):
    
    # we only have activations for T = 10
    path_computed_H = "../data_persisted/activation_maestro_total"
    path_songs = "/Users/***/Downloads/piano_songs/test_songs"
    path_ref = "/Users/***/Downloads/piano_songs/test_midi"
    
    time_limit = 30
    beta = 1
    init = "L1"
    model_AD = True
    note_intensity = "M"
    itmax_H = 100
    tol = 1e-8
    codebook = range(21, 109)
    onset_tolerance = 50/1000
    
    #a = np.arange(1e-3, 1e-2, 1e-3)
    #b = np.arange(1e-4, 1e-3, 1e-4)
    #c = np.arange(1e-5, 1e-4, 1e-5)
    #d = np.arange(1e-6, 1e-5, 1e-6)
    f = np.arange(1e-2, 6e-1, 2e-2)
    #g = np.arange(0.3, 0.5, 0.1)
    #h = np.arange(1, 1.5, 0.1)
    #j = np.arange(1e-7, 1e-6, 1e-7)
    #listthres = np.r_[h[::-1], g[::-1], f[::-1], a[::-1], b[::-1], c[::-1], d[::-1], j[::-1]]
    listthres = np.r_[f[::-1]]

    files = os.listdir(path_songs)
    list_files_wav = []
    for it_files in files:
        if it_files.split(".")[-1] == "wav":
            list_files_wav.append(it_files)

    all_res = []
    for a_song in list_files_wav:
        song_name = a_song.replace(".wav", "")
#         print("processing piano song: {}".format(song_name))
        path_this_song = "{}/{}".format(path_songs, a_song)
        stft = STFT.STFT(path_this_song, time = time_limit, model_AD=model_AD, channel = 0)

        X = stft.get_magnitude_spectrogram()

        annot_name = song_name + ".midi"
        H_persisted_name = "activations_song_{}_W_learned_{}_beta_{}_T_{}_init_{}_stftAD_{}_itmax_{}_intensity_W_{}_time_limit_{}_tol_{}".format(song_name, piano_type, beta, T, init, model_AD, itmax_H, note_intensity, time_limit, tol)
        annot_this_song = "{}/{}".format(path_ref, annot_name)
        note_annotations = load_ref_in_array_midi(annot_this_song, time_limit=time_limit)
        ref = np.array(note_annotations, float)
        ref_pitches = np.array(ref[:,2], int)
        try:
            H = np.load("{}/{}.npy".format(path_computed_H, H_persisted_name), allow_pickle = True)
            res_each_song = []
            for threshold in listthres:
                prediction, _ = tf.transcribe_activations_dynamic(codebook, H, stft, threshold, H_normalization = H_normalization)
                est = np.array(prediction, float)
                if est.size > 0:
                    est_pitches = np.array(est[:,2], int)
                    (prec, rec, f_mes, _) = mir_eval.transcription.precision_recall_f1_overlap(ref[:,0:2], ref_pitches, est[:,0:2], est_pitches, offset_ratio = None, onset_tolerance = onset_tolerance)
                    matching = mir_eval.transcription.match_notes(ref[:,0:2], ref_pitches, est[:,0:2],est_pitches, onset_tolerance=onset_tolerance,offset_ratio=None)
                    TP = len(matching)
                    try:
                        FP = int(TP * (1 - prec) / prec)
                    except ZeroDivisionError:
                        FP = 0
                    try:
                        FN = int(TP * (1 - rec) / rec)
                    except ZeroDivisionError:
                        FN = 0
                    acc = et.accuracy(TP,FP,FN)
                else:
                    prec, rec, f_mes, acc, TP, FP, FN = (0,0,0,0,0,0,0)
                res_each_song.append([prec, rec, f_mes, acc, TP, FP, FN])
            all_res.append(res_each_song)
        except FileNotFoundError:
            print("\033[91m This song failed: {} \033[00m".format(a_song)) 
            pass
        
    np_all_res = np.array(all_res)
    lines = []
    col = ['Precision', 'Recall', 'F measure','Accuracy','True Positives','False Positives','False Negatives']
    for idx, threshold in enumerate(listthres):
        lines.append([round(np.mean(np_all_res[:,idx,i]), 4) for i in range(len(col))])
    df = pd.DataFrame(np.array(lines), columns = col, index = np.round(listthres,decimals = 5))
    display(df.style.bar(subset=["F measure", "Accuracy"], color='#5fba7d'))
    
    best_val = []
    for each_song in range(len(list_files_wav)):
        best_thresh_idx = np.argmax(np_all_res[each_song,:,2])
        best_val.append([round(np_all_res[each_song,best_thresh_idx,i], 4) for i in range(len(col))])
    
    best_df = pd.DataFrame(np.array([np.mean(np.array(best_val)[:,i]) for i in range(len(col))]), columns = ["best res when optimizing thresh"], index = col)
    display(best_df.T)

In [6]:
# intensity M
T = 10
piano_type = "AkPnCGdD"
print('piano_type = ' + piano_type)
compute_scores_database_maestro(piano_type, T, H_normalization = False, adaptative_threshold = True)

piano_type = AkPnCGdD


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
0.59,0.783100,0.205700,0.286300,0.188200,66.322000,18.322000,226.389800
0.57,0.782700,0.215300,0.296800,0.195800,69.480200,20.276800,224.180800
0.55,0.778000,0.225700,0.307800,0.203700,72.836200,22.497200,221.887000
0.53,0.770300,0.237600,0.320400,0.212800,76.565000,25.124300,218.146900
0.51,0.766900,0.249000,0.331500,0.220900,80.254200,28.214700,215.802300
0.49,0.758900,0.261100,0.342800,0.229100,84.146900,31.610200,211.937900
0.47,0.762200,0.273900,0.354900,0.237900,88.248600,35.254200,210.372900
0.45,0.755600,0.287900,0.367600,0.247600,92.757100,39.333300,205.847500
0.43,0.744800,0.302500,0.380000,0.256800,97.435000,44.395500,201.141200
0.41,0.739900,0.318000,0.393400,0.266800,102.395500,49.711900,196.435000


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
best res when optimizing thresh,0.598934,0.592421,0.58761,0.433112,179.80791,149.502825,122.135593


In [7]:
T = 10
piano_type = "ENSTDkCl"
print('piano_type = ' + piano_type)
compute_scores_database_maestro(piano_type, T, H_normalization = False, adaptative_threshold = True)

piano_type = ENSTDkCl


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
0.59,0.762400,0.188200,0.262700,0.168700,60.977400,20.553700,229.491500
0.57,0.757200,0.197200,0.272300,0.175400,63.977400,22.768400,226.497200
0.55,0.751300,0.206600,0.281700,0.181900,67.124300,25.502800,224.271200
0.53,0.752600,0.217400,0.292900,0.189800,70.587600,28.502800,224.197700
0.51,0.752700,0.228700,0.303900,0.197400,74.254200,31.966100,222.016900
0.49,0.748400,0.241000,0.315800,0.205900,78.242900,35.559300,219.062100
0.47,0.739500,0.253800,0.327500,0.214200,82.497200,39.994400,214.779700
0.45,0.728100,0.266500,0.338700,0.222000,86.593200,45.271200,210.632800
0.43,0.717000,0.281200,0.350900,0.230700,91.305100,51.220300,205.926600
0.41,0.705100,0.296700,0.363500,0.239700,96.265500,57.943500,200.966100


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
best res when optimizing thresh,0.561141,0.568802,0.556935,0.399537,170.915254,162.519774,131.090395
